## 자동 주식 데이터 크롤 


### **사용서**


1. 먼저 로그인을 셀을 실행시켜 수동 로그인을 하고 쿠키를 가져온다. 로그인만 하고 input 창에 Enter클릭 후 바로 Chrome 창 끄기 (자동으로 종료 기능 있음 아래 driver.quit() 할 수 있음. 하지만 육안적으로 보고 종료할 거면 주석하리는게 좋음)
   =Cloudflare를 피하기 위함 + 안정적 + 로그인으로 진행할 시 Headless = false 일 경우 Headless가 자주 발생한 경험이 있었음

2. 쿠키를 가져온 상태에서 데이터 수집을 원하는 30개의 기업을 편집한다

3. 3번째 코드를 바로 실행시키면 됨. 

4. (현재 개발중) 첫 파일은 상관없으나 두번째 파일을 다운 받을 때 크롬에서 여러개의 파일을 받을 것 인지 확인한다. 이 때 한번만 허용해주면 된다 하지만 쿠키 기반이기 때문에 코드 실행시 매번 딱 한번만 해줘야 한다. 허용 버튼은 selenium 자동화 설정으로 못 하며 chrome option으로 진행중입니다.

5. 데이터를 계속 잘 다운 받아주면 된다. 

!주의 사항 중간에 실험을 위해 다시 시작 하려고 코드를 중간할 때 경고메세지에 커널 초기화를 누르면 쿠키가 날라가 로그인을 다시 해야 함. 커널 초기화가 되지 않도록 경고메세지를 잘 봐야함 (실험 할 때만 )
!Chromedriver는 본인 로컬에 있는 크롬 버전과 맞춰야 함. github에서 다운 할 수 있음



## Login --dump  cookie

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time

#CloudFlare 우회용 
########절대 건들지 말 것########  
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
########절대 건들지 말 것######## 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
########절대 건들지 말 것######## 

# 로그인 자동화 설정
USERNAME = "hrahn"
PASSWORD = "tndnjseogkrry1234"
wait = WebDriverWait(driver, 20)

try:
    # TradingView 사이트 접속
    driver.get("https://kr.tradingview.com/")
    print("TradingView가 열렸습니다.")
    
    # 1. 로그인 버튼 클릭
    login_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '로그인')]")))
    login_btn.click()
    print("로그인 버튼 클릭 완료")
    time.sleep(2)
    
    # 2. Email 옵션 클릭 (2024/2025 TradingView 필수)
    email_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@name='Email' or .//span[text()='이메일']")))
    email_btn.click()
    print("이메일 로그인 선택 완료")
    time.sleep(1)
    
    # 3. 아이디 입력
    username_field = wait.until(EC.visibility_of_element_located((By.NAME, "username")))
    username_field.send_keys(USERNAME)
    print("아이디 입력 완료")
    time.sleep(0.5)
    
    # 4. 비밀번호 입력
    password_field = driver.find_element(By.NAME, "password")
    password_field.send_keys(PASSWORD)
    print("비밀번호 입력 완료")
    time.sleep(0.5)
    
    # 5. 제출 버튼 클릭
    submit_btn = driver.find_element(By.XPATH, "//button[@type='submit']")
    submit_btn.click()
    print("로그인 버튼 클릭 완료")
    
    # 6. 로그인 완료 대기 (최대 30초)
    time.sleep(5)
    try:
        user_menu = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[class*='user-menu'], [class*='user-id']")))
        print("✓ 로그인 성공!")
    except:
        print("⚠ 로그인 완료 확인 대기 중...")
        time.sleep(3)
        print("✓ 로그인 완료!")
    
    # 로그인이 완료되었는지 확인 (URL 또는 페이지 타이틀 변경)
    time.sleep(2)
    
except Exception as e:
    print(f"로그인 실패: {e}")
    driver.save_screenshot("login_error.png")
    driver.quit()
    raise

# 쿠키 Dump
print("로그인 상태를 저장 중...")
cookies = driver.get_cookies()
with open("tradingview_cookies.pkl", "wb") as f:
    pickle.dump(cookies, f)

print("자동화 작업 수행 가능.")
print(f"현재 페이지: {driver.title}")

driver.quit()


TradingView가 열렸습니다.
로그인 실패: Message: 
Stacktrace:
0   chromedriver                        0x00000001012afdfc cxxbridge1$str$ptr + 3031016
1   chromedriver                        0x00000001012a7cb8 cxxbridge1$str$ptr + 2997924
2   chromedriver                        0x0000000100da2b90 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74192
3   chromedriver                        0x0000000100de9ab4 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 364788
4   chromedriver                        0x0000000100e2aa28 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 630888
5   chromedriver                        0x0000000100dde22c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 317548
6   chromedriver                        0x0000000101274194 cxxbridge1$str$ptr + 2786176
7   chromedriver                        0x0000000101277900 cxxbridge1$str$ptr + 2800364
8   chromedriver                        0x000000010125442c cxxb

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001012afdfc cxxbridge1$str$ptr + 3031016
1   chromedriver                        0x00000001012a7cb8 cxxbridge1$str$ptr + 2997924
2   chromedriver                        0x0000000100da2b90 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74192
3   chromedriver                        0x0000000100de9ab4 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 364788
4   chromedriver                        0x0000000100e2aa28 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 630888
5   chromedriver                        0x0000000100dde22c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 317548
6   chromedriver                        0x0000000101274194 cxxbridge1$str$ptr + 2786176
7   chromedriver                        0x0000000101277900 cxxbridge1$str$ptr + 2800364
8   chromedriver                        0x000000010125442c cxxbridge1$str

## 주식 리스트 30개 넣기

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pickle
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [4]:
STOCK_LIST = [
    "BTCUSD",           # 비트코인
    "ETHUSD",           # 이더리움
    "XRPUSD",           # 리플
    "DOGEUSD",          # 도지코인
    "SOLUSD",           # 솔라나

    "GOOGL",            # 구글
    "AAPL",             # 애플
    "MSFT",             # 마이크로소프트
    "AMZN",             # 아마존
    "TSLA",             # 테슬라
    "NVDA",             # 엔비디아
    "META",             # 메타(페이스북)
    "NFLX",             # 넷플릭스
    "005930",           # 삼성전자
    "000660",           # SK하이닉스

    "BABA",             # 알리바바
    "TSM",              # TSMC
    "AMD",              # AMD
    "INTC",             # 인텔
    "ORCL",             # 오라클
    "IBM",              # IBM
    "V",                # 비자
    "MA",               # 마스터카드
    "JPM",              # JP모건
    "GS",               # 골드만삭스
    "KO",               # 코카콜라
    "PEP",              # 펩시
    "NKE",              # 나이키
    "DIS",              # 디즈니
    "Trump",            # 트럼프코인

    # 나스닥 상위 70개 추가 (기존 30개 제외)
    "AVGO",             # 브로드컴
    "COST",             # 코스트코
    "ADBE",             # 어도비
    "CMCSA",            # 컴캐스트
    "QCOM",             # 퀄컴
    "TXN",              # 텍사스인스트루먼츠
    "CSCO",             # 시스코
    "INTU",             # 인투잇
    "ASML",             # ASML
    "AZN",              # 애스트라제네카
    "GILD",             # 길리어드
    "CHTR",             # 차터커뮤니케이션스
    "SNPS",             # 시놉시스
    "CDNS",             # 캐던스 디자인
    "PAYX",             # 페이첵스
    "MDLZ",             # 몬델레즈
    "KLAC",             # KLA
    "AMAT",             # 어플라이드 머터리얼즈
    "PEG",              # 퍼블릭 서비스 엔터프라이즈
    "TMUS",             # T-모바일
    "LRCX",             # LAM 리서치
    "MCHP",             # 마이크로칩 테크놀로지
    "ADP",              # 오토매틱 데이터 프로세싱
    "HON",              # 허니웰
    "ADSK",             # 오토데스크
    "VRSK",             # 베리사인
    "JBHT",             # JB 헌트
    "EXC",              # 엑셀론
    "XEL",              # 엑셀 에너지
    "NXPI",             # NXP 세미컨덕터
    "REGN",             # 리제너론
    "PYPL",             # 페이팔
    "CTSH",             # 코그니잔트
    "ANSS",             # ANSYS
    "BIIB",             # 바이오젠
    "VRTX",             # 버텍스 파마슈티컬즈
    "PCAR",             # 팩카
    "CPRT",             # 코파트
    "ALNY",             # 알나일람
    "IDXX",             # IDEXX 래버래토리즈
    "FAST",             # 파스테날
    "WBA",              # 월그린 부츠 얼라이언스
    "AMD",              # AMD (재확인)
]

# 전세계 시가총액 TOP 200 중 기존 STOCK_LIST에 없는 순차적 100개
STOCK_LIST = [
    "BRK.B",            # 버크셔 해서웨이
    "JNJ",              # 존슨앤존슨
    "WMT",              # 월마트
    "PG",               # 프록터 앤 갬블
    "ABBV",             # 애브비
    "XOM",              # 엑손모빌
    "MCD",              # 맥도날드
    "UNH",              # 유나이티드 헬스그룹
    "LLY",              # 일라이 릴리
    "MRK",              # 머크
    "CVX",              # 쉐브론
    "PFE",              # 화이자
    "ABT",              # 애보트
    "WM",               # 웨이스트 매니지먼트
    "TMO",              # 써모 피셔 사이언티픽
    "KMB",              # 킴벌리클라크
    "BAC",              # 뱅크 오브 아메리카
    "AXP",              # 아메리칸 익스프레스
    "VZ",               # 버라이존
    "CAT",              # 캐터필러
    "COP",              # 코노코필립스
    "BLK",              # 블랙록
    "DE",               # 존디어
    "WFC",              # 웰스파고
    "T",                # AT&T
    "RTX",              # 레이시온 테크놀로지
    "GE",               # 제너럴 일렉트릭
    "SCHW",             # 찰스 슈웹
    "F",                # 포드
    "BA",               # 보잉
    "GIS",              # 제너럴 밀스
    "MMM",              # 3M
    "MO",               # 필립 모리스
    "UPS",              # UPS
    "BSX",              # 보스턴 사이언티픽
    "CB",               # 쿠퍼 스탠다드
    "ADI",              # 애널로그 디바이시즈
    "FDX",              # FedEx
    "GM",               # 제너럴 모터스
    "SO",               # 더 서던 컴퍼니
    "NEE",              # 넥스트에라 에너지
    "BK",               # 뉴욕 멜론 은행
    "TJX",              # TJ 맥스
    "SYK",              # 스트라이커
    "LMT",              # 록히드 마틴
    "ELV",              # 엘리베이션
    "STLA",             # 스텔란티스
    "EMR",              # 에머슨 일렉트릭
    "PLD",              # 포라이트 인더스트리얼
    "IQV",              # IQVIA
    "RCL",              # 로열 캐리비언 크루즈
    "PM",               # 필립 모리스 인터내셔널
    "CI",               # 시그나
    "SAN",              # 사노피
    "CP",               # 캐나디언 퍼시픽
    "CNP",              # 센트로포인트 에너지
    "DHI",              # D.R. 호턴
    "MCO",              # 무디스
    "EQIX",             # 이쿠익스
    "DRI",              # 다이닝 리소르트 인터내셔널
    "GD",               # 제너럴 다이나믹스
    "HUM",              # 휴매나
    "HD",               # 홈 디포
    "PWR",              # 파워 인테그레이션스
    "SRE",              # 세마 에너지
    "ROK",              # 로크웰 오토메이션
    "MSCI",             # MSCI
    "DUK",              # 듀크 에너지
    "ITW",              # 일리노이스 툴 웍스
    "LEN",              # 렌너 인터내셔널
    "NRG",              # NRG 에너지
    "ENPH",             # 엔페이즈 에너지
    "MPC",              # 마라톤 페트롤리움
    "SBAC",             # SBA 커뮤니케이션
    "MAR",              # 메리어트
    "LUV",              # 사우스웨스트 에어라인
    "HAL",              # 할리버튼
    "APD",              # 에어 프로덕츠
    "AVY",              # 에버리
    "BXP",              # 보스턴 프로퍼티즈
    "AZO",              # 오토존
    "WELL",             # 웰타워
    "OKE",              # 원로크 에너지
    "PSA",              # 퍼블릭 스토리지
    "HLT",              # 힐튼
    "CEG",              # 컴스테드 에너지
    "EIX",              # 에디슨 인터내셔널
    "DAL",              # 델타 에어라인
    "PSX",              # 필립스 66
    "REG",              # 리전시 센터
    "VLO",              # 발레로 에너지
]



STOCK_LIST =  [
    "005930", "035420", "035720", "000660", "034020", "042660", "005380", "006400", "086520", "373220",
    "012450", "068270", "042700", "247540", "000720", "108490", "001440", "010140", "005490", "272210",
    "064350", "032500", "015760", "051910", "003670", "066570", "352820", "066970", "011790", "329180",
    "010120", "336260", "000270", "196170", "009830", "278470", "079550", "097230", "032820", "377300",
    "277810", "000150", "003230", "207940", "483650", "028260", "007660", "090710", "298040", "347850",
    "052690", "011200", "036570", "267260", "454910", "006800", "047810", "067310", "018260", "005935",
    "035900", "009540", "064400", "003490", "352480", "103590", "119850", "003850", "005690", "474170",
    "096770", "323410", "257720", "466100", "009150", "090360", "047050", "402340", "050890", "214450",
    "308080", "222800", "034220", "000250", "218410", "103140", "259960", "310210", "028300", "082740",
    "062040", "476040", "456010", "010130", "090430", "030530", "348370", "375500", "298380", "046120",
    "000880", "083650", "105560", "010620", "017670", "000100", "128940", "058470", "018880", "240810",
    "160190", "011070", "316140", "226950", "064260", "126340", "114190", "087010", "100090", "041190",
    "012330", "064550", "382900", "378800", "022100", "042670", "141080", "161580", "017960", "071970",
    "025980", "034730", "086790", "039490", "300080", "267270", "005070", "450080", "326030", "080220",
    "214150", "241560", "001500", "004370", "117730", "000500", "010170", "075580", "068760", "008970",
    "010950", "051900", "138080", "088800", "100840", "437730", "003160", "060370", "361610", "089030",
    "489790", "468530", "041510", "122870", "417200", "348210", "353200", "138040", "304100", "093370",
    "237690", "006340", "145020", "006260", "004020", "065350", "033100", "054540", "003570", "033780",
    "098460", "084370", "105840", "302440", "112610", "356860", "001040", "069540", "195870", "199430",
    "071050", "192820", "293490", "077970", "089010", "356680", "014940", "403870", "055550", "475400",
    "267250", "028050", "051600", "036930", "419530", "161890", "032830", "125490", "036460", "018290",
    "253590", "290650", "139480", "475830", "123690", "035250", "131970", "011210", "457190", "051980",
    "271560", "005950", "014680", "278280", "056080", "078600", "439260", "389020", "307950", "060250",
    "029480", "328130", "042000", "101490", "010060", "380550", "039030", "263750", "011170", "440110",
    "161390", "376300", "031980", "086390", "086280", "357780", "322000", "214370", "036810", "089890"
]

    # "402030", "005290", "019170", "005385", "097950", "388720", "000990", "178320", "053950", "253450",
    # "389470", "003550", "340930", "033500", "140860", "229640", "251270", "204320", "091970", "095610",
    # "462870", "250930", "017800", "287840", "124500", "455900", "005880", "443060", "460930", "028670",
    # "036620", "045390", "005387", "016360", "445680", "008770", "356890", "005850", "452190", "047770",
    # "252990", "319660", "108320", "232140", "249420", "388050", "003280", "030200", "044490", "195940",
    # "298050", "137400", "006360", "107640", "383310", "234340", "098070", "213420", "413630", "082270",
    # "030520", "033340", "199820", "215100", "052400", "009520", "225190", "267320", "074600", "058610",
    # "096530", "183300", "000810", "095500", "002960", "006910", "121600", "432720", "089970", "204620",
    # "456160", "032350", "099320", "333430", "416180", "000440", "047560", "145720", "222080", "007390",
    # "014620", "256840", "391710", "003960", "381620", "092460", "108860", "336370", "020150", "200670",
    # "460940", "006040", "019990", "112040", "309710", "067160", "079940", "458870", "011780", "281740",
    # "376900", "180640", "241710", "039200", "047920", "064760", "101670", "012510", "094820", "230240",
    # "322780", "457550", "476060", "317330", "009420", "285130", "008930", "077360", "399720", "475150",
    # "140410", "456040", "005940", "014950", "023590", "338220", "032640", "035760", "083450", "092730",
    # "015750", "037270", "092870", "358570", "009240", "005420", "064820", "095340", "136150", "003620",
    # "049080", "085660", "090370", "024110", "131290", "377480", "475960", "204270", "002380", "271940",
    # "397030", "161000", "225570", "460860", "004800", "034230", "085310", "194700", "248070", "073490",
    # "127120", "270660", "001430", "007810", "317450", "000080", "027050", "032190", "051780", "084110",
    # "090460", "424980", "069960", "215790", "000155", "001720", "179900", "018000", "047040", "166090",
    # "251970", "310200", "445090", "463480", "475560", "001530", "004170", "005180", "008730", "040910",
    # "099440", "295310", "024940", "065440", "171090", "340570", "004990", "023790", "062970", "100590",
    # "260970", "003530"
print(len(STOCK_LIST))

250


In [5]:
# 전세계 TOP 200 순차적 200개 종목
STOCK_LIST = [
    "BRK.B",            # 버크셔 해서웨이
    "JNJ",              # 존슨앤존슨
    "WMT",              # 월마트
    "PG",               # 프록터 앤 갬블
    "ABBV",             # 애브비
    "XOM",              # 엑손모빌
    "MCD",              # 맥도날드
    "UNH",              # 유나이티드 헬스그룹
    "LLY",              # 일라이 릴리
    "MRK",              # 머크
    "CVX",              # 쉐브론
    "PFE",              # 화이자
    "ABT",              # 애보트
    "WM",               # 웨이스트 매니지먼트
    "TMO",              # 써모 피셔 사이언티픽
    "KMB",              # 킴벌리클라크
    "BAC",              # 뱅크 오브 아메리카
    "AXP",              # 아메리칸 익스프레스
    "VZ",               # 버라이존
    "CAT",              # 캐터필러
    "COP",              # 코노코필립스
    "BLK",              # 블랙록
    "DE",               # 존디어
    "WFC",              # 웰스파고
    "T",                # AT&T
    "RTX",              # 레이시온 테크놀로지
    "GE",               # 제너럴 일렉트릭
    "SCHW",             # 찰스 슈웹
    "F",                # 포드
    "BA",               # 보잉
    "GIS",              # 제너럴 밀스
    "MMM",              # 3M
    "MO",               # 필립 모리스
    "UPS",              # UPS
    "BSX",              # 보스턴 사이언티픽
    "CB",               # 쿠퍼 스탠다드
    "ADI",              # 애널로그 디바이시즈
    "FDX",              # FedEx
    "GM",               # 제너럴 모터스
    "SO",               # 더 서던 컴퍼니
    "NEE",              # 넥스트에라 에너지
    "BK",               # 뉴욕 멜론 은행
    "TJX",              # TJ 맥스
    "SYK",              # 스트라이커
    "LMT",              # 록히드 마틴
    "ELV",              # 엘리베이션
    "STLA",             # 스텔란티스
    "EMR",              # 에머슨 일렉트릭
    "PLD",              # 포라이트 인더스트리얼
    "IQV",              # IQVIA
    "RCL",              # 로열 캐리비언 크루즈
    "PM",               # 필립 모리스 a
    "CI",               # 시그나
    "SAN",              # 사노피
    "CP",               # 캐나디언 퍼시픽
    "CNP",              # 센트로포인트 에너지
    "DHI",              # D.R. 호턴
    "MCO",              # 무디스
    "EQIX",             # 이쿠익스
    "DRI",              # 다이닝 리소르트 인터내셔널
    "GD",               # 제너럴 다이나믹스
    "HUM",              # 휴매나
    "HD",               # 홈 디포
    "PWR",              # 파워 인테그레이션스
    "SRE",              # 세마 에너지
    "ROK",              # 로크웰 오토메이션
    "MSCI",             # MSCI
    "DUK",              # 듀크 에너지
    "ITW",              # 일리노이스 툴 웍스
    "LEN",              # 렌너 인터내셔널
    "NRG",              # NRG 에너지
    "ENPH",             # 엔페이즈 에너지
    "MPC",              # 마라톤 페트롤리움
    "SBAC",             # SBA 커뮤니케이션
    "MAR",              # 메리어트
    "LUV",              # 사우스웨스트 에어라인
    "HAL",              # 할리버튼
    "APD",              # 에어 프로덕츠
    "AVY",              # 에버리
    "BXP",              # 보스턴 프로퍼티즈
    "AZO",              # 오토존
    "WELL",             # 웰타워
    "OKE",              # 원로크 에너지
    "PSA",              # 퍼블릭 스토리지
    "HLT",              # 힐튼
    "CEG",              # 컴스테드 에너지
    "EIX",              # 에디슨 인터내셔널
    "DAL",              # 델타 에어라인
    "PSX",              # 필립스 66
    "REG",              # 리전시 센터
    "VLO",              # 발레로 에너지
    "ELAB",             # ELAB
    "PLTR",             # Palantir Technologies Inc.
    "IONQ",             # IonQ, Inc.
    "CRCL",             # Circle Internet Group, Inc.
    "BMNR",             # BitMine Immersion Technologies, Inc.
    "MSTR",             # Strategy Inc
    "RGTI",             # Rigetti Computing, Inc.
    "SMR",              # NuScale Power Corporation
    "OKLO",             # Oklo Inc.
    "COIN",             # Coinbase Global, Inc.
    "BYND",             # Beyond Meat, Inc.
    "HOOD",             # Robinhood Markets, Inc.
    "SOFI",             # SoFi Technologies, Inc.
    "RKLB",             # Rocket Lab Corporation
    "CRWV",             # CoreWeave, Inc.
    "LAES",             # SEALSQ Corp
    "JOBY",             # Joby Aviation, Inc.
    "TEM",              # Tempus AI, Inc.
    "NBIS",             # Nebius Group N.V.
    "MP",               # MP Materials Corp.
    "QCOM",             # QUALCOMM Incorporated
    "AVGO",             # Broadcom Inc.
    "NFLX",             # Netflix, Inc.
    "QBTS",             # D-Wave Quantum Inc.
    "MU",               # Micron Technology, Inc.
    "OPEN",             # Opendoor Technologies Inc
    "RDDT",             # Reddit, Inc.
    "SMCI",             # Super Micro Computer, Inc.
    "RXRX",             # Recursion Pharmaceuticals, Inc.
    "NVO",              # Novo Nordisk A/S
    "ASTS",             # AST SpaceMobile, Inc.
    "PGY",              # Pagaya Technologies Ltd.
    "FIG",              # FIGMA INC
    "LEU",              # Centrus Energy Corp.
    "PLUG",             # Plug Power, Inc.
    "LULU",             # lululemon athletica inc.
    "U",                # Unity Software Inc.
    "GME",              # GameStop Corporation
    "CRDO",             # Credo Technology Group Holding Ltd
    "CRML",             # Critical Metals Corp.
    "CRM",              # Salesforce, Inc.
    "CPNG",             # Coupang, Inc.
    "MRVL",             # Marvell Technology, Inc.
    "OXY",              # Occidental Petroleum Corporation
    "QS",               # QuantumScape Corporation
    "RZLV",             # Rezolve AI PLC
    "ARM",              # ARM Holdings PLC
    "ABNB",             # Airbnb, Inc.
    "ONDS",             # Ondas Holdings Inc.
    "INOD",             # Innodata Inc.
    "VST",              # Vistra Corp.
    "UPST",             # Upstart Holdings, Inc.
    "UMAC",             # Unusual Machines, Inc.
    "ACHR",             # Archer Aviation Inc.
    "ASML",             # ASML Holding NV
    "LAC",              # Lithium Americas Corp.
    "SYM",              # Symbotic Inc.
    "NTLA",             # Intellia Therapeutics, Inc.
    "IBIT",             # iShares Bitcoin Trust ETF
    "ZETA",             # Zeta Global Holdings Corp.
    "LUNR",             # Intuitive Machines, Inc.
    "SBET",             # SharpLink Gaming, Inc.
    "RIVN",             # Rivian Automotive, Inc.
    "LCID",             # Lucid Group, Inc.
    "CRSP",             # CRISPR Therapeutics AG
    "SLDP",             # Solid Power, Inc.
    "USAR",             # USA Rare Earth, Inc.
    "CRWD",             # CrowdStrike Holdings, Inc.
    "COST",             # Costco Wholesale Corporation
    "CLSK",             # CleanSpark, Inc.
    "SNPS",             # Synopsys, Inc.
    "ISRG",             # Intuitive Surgical, Inc.
    "APLD",             # Applied Digital Corporation
]


STOCK_LIST = [
    "NVDA", "AAPL", "MSFT", "AMZN", "AVGO", "GOOGL", "META", "GOOG", "TSLA", "BRK.B",
    "JPM", "WMT", "LLY", "ORCL", "V", "MA", "XOM", "PLTR", "NFLX", "JNJ",
    "AMD", "COST", "ABBV", "BAC", "HD", "PG", "GE", "UNH", "CVX", "KO",
    "IBM", "CSCO", "CAT", "WFC", "MS", "MU", "AXP", "CRM", "GS", "RTX",
    "TMUS", "PM", "ABT", "MCD", "MRK", "TMO", "APP", "LRCX", "PEP", "LIN",
    "UBER", "DIS", "ANET", "NOW", "INTC", "QCOM", "ISRG", "AMAT", "INTU", "C",
    "T", "SCHW", "BLK", "APH", "NEE", "BKNG", "VZ", "KLAC", "TJX", "AMGN",
    "GEV", "ACN", "DHR", "BA", "SPGI", "ETN", "BSX", "PANW", "GILD", "TXN",
    "ADBE", "SYK", "COF", "PFE", "CRWD", "LOW", "UNP", "HON", "DE", "WELL",
    "HOOD", "PGR", "CEG", "MDT", "BX", "PLD", "ADI", "LMT", "COP", "CB",
    "HCA", "DASH", "DELL", "VRTX", "ADP", "KKR", "SO", "MCK", "CMCSA", "TT",
    "PH", "DUK", "CVS", "MO", "NKE", "CME", "SBUX", "GD", "BMY", "CDNS",
    "NEM", "MMM", "MMC", "SHW", "MCO", "ICE", "COIN", "AMT", "NOC", "SNPS",
    "EQIX", "HWM", "WM", "UPS", "ORLY", "RCL", "ABNB", "EMR", "BK", "MDLZ",
    "JCI", "CTAS", "TDG", "GLW", "ECL", "USB", "MSI", "TEL", "ELV", "PNC",
    "MAR", "AON", "ITW", "APO", "WMB", "REGN", "PWR", "AJG", "CSX", "COR",
    "CI", "RSG", "AEP", "MNST", "FTNT", "VST", "ADSK", "GM", "PYPL", "ZTS",
    "NSC", "WDAY", "CL", "AZO", "HLT", "TRV", "CMI", "SRE", "FCX", "MPC",
    "FDX", "DLR", "KMI", "AXON", "EOG", "STX", "AFL", "TFC", "SPG", "PSX",
    "LHX", "DDOG", "URI", "APD", "SLB", "WBD", "O", "MET", "MPWR", "F",
    "NXPI", "PCAR", "VLO", "BDX", "ROST", "D", "ALL", "IDXX", "CARR", "EA",
    "PSA", "EW", "NDAQ", "XEL", "BKR", "ROP", "FAST", "EXC", "WDC", "TTWO",
    "AME", "GWW", "CBRE", "CAH", "XYZ", "DHI", "MSCI", "CMG", "KR", "AIG",
    "ETR", "AMP", "TGT", "PAYX", "OKE", "CTVA", "GRMN", "FANG", "CPRT", "ROK",
    "A", "PEG", "OXY", "KMB", "LVS", "CCI", "YUM", "VMC", "EBAY", "FICO",
    "KDP", "DAL", "CCL", "MLM", "XYL", "IQV", "RMD", "OTIS", "PRU", "WEC"
]

print(len(STOCK_LIST))

260


## Detail Options 수정 금지

In [6]:
TIME_PERIODS = [
    {
        "name": "12달",
        "button_text": "연",
        "option_xpath": "//span[contains(@class, 'label-jFqVJoPk') and text()='12 달']",
        "need_scroll": False,
        "scroll_seconds": 0
    },
    {
        "name": "1달",
        "button_text": "달",
        "option_xpath": "//span[contains(@class, 'label-jFqVJoPk') and text()='1 달']",
        "need_scroll": False,
        "scroll_seconds": 0
    },
    {
        "name": "1주",
        "button_text": "주",
        "option_xpath": "//span[contains(@class, 'label-jFqVJoPk') and text()='1 주']",
        "need_scroll": True,
        "scroll_seconds":2  
    },
    {
        "name": "1일",
        "button_text": "날",
        "option_xpath": "//span[contains(@class, 'label-jFqVJoPk') and text()='1 날']",
        "need_scroll": True,
        "scroll_seconds": 3 
    },
    {
        "name": "1시간",
        "button_text": "1시간",
        "option_xpath": "//span[contains(@class, 'label-jFqVJoPk') and text()='1 시간']",
        "need_scroll": True,
        "scroll_seconds": 25
    },
    {
        "name": "10분",
        "button_text": "10분",
        "option_xpath": "//span[contains(@class, 'label-jFqVJoPk') and text()='10 분']",
        "need_scroll": True,
        "scroll_seconds": 10 
    }
]

## 찐찐

In [8]:
print("자동화 시작 ! 이제 더 이상 건드리지 않아도 됩니다.")
chrome_driver_path = "/opt/homebrew/bin/chromedriver"

options = Options()
#options.add_argument("--headless=new")  
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--force-device-scale-factor=0.25")
download_dir = os.path.join(os.getcwd(), "tradingview_data")
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "profile.default_content_setting_values.automatic_downloads": 1,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
}
options.add_experimental_option("prefs", prefs)

os.makedirs(download_dir, exist_ok=True)
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=options)

driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd(
    "Page.setDownloadBehavior",
    {
        "behavior": "allow",
        "downloadPath": download_dir
    }
)

wait = WebDriverWait(driver, 30)

def safe_click(element_finder, description, wait_time=0.5):
    try:
        print(f"  → {description} 시도 중...")
        element = element_finder()
        time.sleep(0.3)
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)

        
        driver.execute_script("arguments[0].click();", element)
        print(f"  {description} 성공!")
        time.sleep(wait_time)
        return True
    except Exception as e:
        try:
            element = element_finder()
            actions = ActionChains(driver)
            actions.move_to_element(element).click().perform()
            print(f"  {description} 성공! (ActionChains)")
            time.sleep(wait_time)
            return True
        except:
            print(f"  {description} 실패")
            return False

def chart_scroll_left_continuous(seconds):
    
    try:
        chart_area = driver.find_element(By.CSS_SELECTOR, "div.chart-container, canvas, div[class*='chart']")
        chart_area.click()
    except:
        driver.find_element(By.TAG_NAME, "body").click()
    
    print(f"  → 차트 스크롤 시작 ({seconds}초간)...")
    
    actions = ActionChains(driver)
    time.sleep(2.5)

    start_time = time.time()
    actions.key_down(Keys.COMMAND).perform()
    
    while time.time() - start_time < seconds:
        actions.send_keys(Keys.LEFT).perform()
        time.sleep(1.1)
    
    actions.key_up(Keys.COMMAND).perform()
    
    print(f"  차트 스크롤 완료!")
    time.sleep(0.5) 

def search_and_select_stock(stock_name):
    """종목 검색 및 선택"""
    print(f"\n[{stock_name}] 검색 시작")
    
    try:
        time.sleep(0.5)
        search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
            "button.tv-header-search-container")))
        search_button.click()
        print(f"검색 버튼 클릭")
        
        time.sleep(0.1)
        
        print(f"'{stock_name}' 입력 중")
        actions = ActionChains(driver)
        actions.key_down(Keys.COMMAND).send_keys('a').key_up(Keys.COMMAND).perform()
        time.sleep(0.2)
        actions.send_keys(Keys.DELETE).perform()
        time.sleep(0.2)
        actions.send_keys(stock_name).perform()
        print(f"검색어 입력 완료")
        
        time.sleep(0.5)
        
        actions.send_keys(Keys.ENTER).perform()
        print(f"종목 선택 완료")
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"검색/선택 실패: {e}")
        return False

def export_data_for_period(period_info):
    time.sleep(3)
    period_name = period_info["name"]
    print(f"\n  [{period_name}] 익스포트 프로세스 시작")
    
    try:
        def find_period_button():
            return driver.find_element(By.CSS_SELECTOR, "#header-toolbar-intervals > button > div > div")
        
        if not safe_click(find_period_button, f"{period_name} - 시간 단위 메뉴 열기"):
            return False
        
        def find_period_option():
            return driver.find_element(By.XPATH, period_info["option_xpath"])
        
        if not safe_click(find_period_option, f"{period_name} 선택"):
            return False
        
        if period_info["need_scroll"]:
            chart_scroll_left_continuous(period_info["scroll_seconds"])
    
        def find_arrow_button():
            try:
                arrows = driver.find_elements(By.CSS_SELECTOR, "div.arrow-merBkM5y")
                if len(arrows) >= 14:
                    return arrows[13]  
                else:
                    return driver.find_element(By.CSS_SELECTOR, "body > div.js-rootresizer__contents > div > div.layout__area--top > div > div > div:nth-child(3) > div.wrapOverflow-wXGVFOC9 > div > div > div > div > div:nth-child(14) > div > div > div > button > div")
            except:
                return driver.find_element(By.XPATH, "//div[contains(@class, 'arrow-merBkM5y')]")
        
        safe_click(find_arrow_button, " 더보기 화살표 클릭")
        
        def find_export_option():
            try:
                return driver.find_element(By.CSS_SELECTOR, "#overlap-manager-root > div:nth-child(2) > span > div.menu-yyMUOAN9.menuWrap-Kq3ruQo8 > div > div > div:nth-child(6) > span.labelRow-jFqVJoPk > span")
            except:
                return driver.find_element(By.XPATH, "//span[contains(@class, 'label-jFqVJoPk') and contains(text(), '익스포트 차트 데이터')]")
        
        safe_click(find_export_option, "익스포트 차트 데이터 클릭")
        
        def find_unix_timestamp():
            try:
                return driver.find_element(By.CSS_SELECTOR, "#time-format-select > span.inner-slot-W53jtLjw.inner-middle-slot-W53jtLjw > span")
            except:
                return driver.find_element(By.XPATH, "//span[contains(@class, 'button-children-tFul0OhX')]/span[text()='UNIX 타임스탬프']")
        
        safe_click(find_unix_timestamp, " UNIX 타임스탬프 클릭")
        
        def find_iso_time():
            try:
                return driver.find_element(By.CSS_SELECTOR, "#time-format-iso > span")
            except:
                return driver.find_element(By.XPATH, "//span[contains(@class, 'label-jFqVJoPk') and text()='ISO 타임']")
        
        safe_click(find_iso_time, "ISO 타임 선택")
        
        def find_export_button():
            try:
                return driver.find_element(By.CSS_SELECTOR, "#overlap-manager-root > div:nth-child(2) > div > div.dialog-qyCw0PaN.dialog-b8SxMnzX.dialog-aRAWUDhF.rounded-aRAWUDhF.shadowed-aRAWUDhF > div > div.footer-PhMf7PhQ > div > span > button")
            except:
                try:
                    return driver.find_element(By.XPATH, "//button[@name='submit' and contains(@class, 'button-D4RPB3ZC')]//span[text()='익스포트']")
                except:
                    return driver.find_element(By.XPATH, "//span[contains(@class, 'content-D4RPB3ZC') and text()='익스포트']/parent::button")
        
        safe_click(find_export_button, "익스포트 버튼 클릭")
        
        print(f"  [{period_name}] 익스포트 완료!")
        return True
        
    except Exception as e:
        print(f"  [{period_name}] 익스포트 실패: {e}")
        return False

def process_single_stock(stock_name, idx, total_stocks):
    print(f"\n{'='*60}")
    print(f"[{idx}/{total_stocks}] {stock_name} 처리 시작 ({round(idx/total_stocks*100)}%)")
    print(f"{'='*60}")
    
    driver.get("https://kr.tradingview.com/")
    time.sleep(1)
    
    if not search_and_select_stock(stock_name):
        return False
    
    time.sleep(4)
    
    success_count = 0
    for i, period in enumerate(TIME_PERIODS):
        print(f"\n  처리 순서: {i+1}/{len(TIME_PERIODS)} | {stock_name} - {period['name']} 크롤링 중...")
        if export_data_for_period(period):  
            success_count += 1
        time.sleep(1)
    
    print(f"\n✓ [{idx}/{total_stocks}] {stock_name} 완료: {success_count}/{len(TIME_PERIODS)} 익스포트 성공")
    return success_count > 0

# ========== 메인 실행 ==========
total_success = 0
total_fail = 0

try:
    print("\n" + "="*60)
    print("TradingView 자동 크롤링 시작!")
    print("="*60 + "\n")
    
    driver.get("https://kr.tradingview.com/")
    
    try:
        with open("tradingview_cookies.pkl", "rb") as f:
            cookies = pickle.load(f)
        for cookie in cookies:
            try:
                driver.add_cookie(cookie)
            except:
                pass
        print("✓ 쿠키 로드 완료\n")
    except:
        print("쿠키 없이 진행\n")
    
    time.sleep(1)
    
    for idx, stock in enumerate(STOCK_LIST, 1):
        if process_single_stock(stock, idx, len(STOCK_LIST)):
            total_success += 1
        else:
            total_fail += 1
        
    
    print(f"\n{'='*60}")
    print(f"전체 작업 완료!")
    print(f"성공: {total_success}/{len(STOCK_LIST)}, 실패: {total_fail}/{len(STOCK_LIST)}")
    print(f"완료율: {round(total_success/len(STOCK_LIST)*100)}%")
    print(f"저장 위치: {download_dir}")
    print(f"{'='*60}")

except Exception as e:
    print(f"\n오류 발생: {e}")
    import traceback
    traceback.print_exc()

finally:
    print("\n프로그램 종료 중...")
    time.sleep(2)
    driver.quit()
    print("✓ 완료!")

자동화 시작 ! 이제 더 이상 건드리지 않아도 됩니다.

TradingView 자동 크롤링 시작!

✓ 쿠키 로드 완료


[1/260] NVDA 처리 시작 (0%)

[NVDA] 검색 시작
검색 버튼 클릭
'NVDA' 입력 중
검색어 입력 완료
종목 선택 완료

  처리 순서: 1/6 | NVDA - 12달 크롤링 중...

  [12달] 익스포트 프로세스 시작
  → 12달 - 시간 단위 메뉴 열기 시도 중...
  12달 - 시간 단위 메뉴 열기 성공!
  → 12달 선택 시도 중...
  12달 선택 성공!
  →  더보기 화살표 클릭 시도 중...
   더보기 화살표 클릭 성공!
  → 익스포트 차트 데이터 클릭 시도 중...
  익스포트 차트 데이터 클릭 성공!
  →  UNIX 타임스탬프 클릭 시도 중...
   UNIX 타임스탬프 클릭 성공!
  → ISO 타임 선택 시도 중...
  ISO 타임 선택 성공!
  → 익스포트 버튼 클릭 시도 중...
  익스포트 버튼 클릭 실패
  [12달] 익스포트 완료!

  처리 순서: 2/6 | NVDA - 1달 크롤링 중...

  [1달] 익스포트 프로세스 시작
  → 1달 - 시간 단위 메뉴 열기 시도 중...
  1달 - 시간 단위 메뉴 열기 성공!
  → 1달 선택 시도 중...
  1달 선택 성공!
  →  더보기 화살표 클릭 시도 중...
   더보기 화살표 클릭 성공!
  → 익스포트 차트 데이터 클릭 시도 중...
  익스포트 차트 데이터 클릭 실패
  →  UNIX 타임스탬프 클릭 시도 중...
   UNIX 타임스탬프 클릭 성공!
  → ISO 타임 선택 시도 중...
  ISO 타임 선택 성공!
  → 익스포트 버튼 클릭 시도 중...
  익스포트 버튼 클릭 실패
  [1달] 익스포트 완료!

  처리 순서: 3/6 | NVDA - 1주 크롤링 중...

  [1주] 익스포트 프로세스 시작
  → 1주 - 시간 단위 메뉴 열기 시도 중...
  1주 - 시간 단위 메뉴 열기 성공!
 

KeyboardInterrupt: 

In [8]:
import os
import re
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
from sshtunnel import SSHTunnelForwarder
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# ====== 설정 ======
CSV_DIR = r"C:\Users\wntjd\데이터크롤링_공모\tradingview_data"
DB_NAME = "etf1_db"

SSH_CONFIG = {
    "ssh_address_or_host": "ahnbi2.suwon.ac.kr",
    "ssh_username": "ahnbi2",
    "ssh_password": "bigdata",
    "ssh_port": 22,
    "remote_bind_address": ("127.0.0.1", 5100)
}

MYSQL_CONFIG = {
    "user": "etf1",
    "password": "deepdata",
    "host": "127.0.0.1",
    "port": 5100
}

OVERWRITE = True
# =================

def safe_table_name(name):
    """테이블명 안전하게 변환: 특수문자 -> '_' """
    return re.sub(r'\W+', '_', name)

def connect_mysql(tunnel):
    """MySQL 서버 연결"""
    try:
        MYSQL_CONFIG["port"] = tunnel.local_bind_port
        conn = mysql.connector.connect(**MYSQL_CONFIG)
        cursor = conn.cursor()
        return conn, cursor
    except mysql.connector.Error as err:
        print(f"MySQL 연결 실패: {err}")
        exit(1)

def create_database(cursor):
    """데이터베이스 생성 (없으면 생성)"""
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{DB_NAME}` DEFAULT CHARACTER SET 'utf8mb4'")
    except mysql.connector.Error as err:
        print(f"데이터베이스 생성 실패: {err}")
        exit(1)

def create_table_from_csv(cursor, table_name, df):
    """CSV 헤더 기반 테이블 생성"""
    columns = ", ".join([f"`{col}` TEXT" for col in df.columns])
    cursor.execute(f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})")

def insert_data(cursor, conn, table_name, df):
    """CSV 데이터를 테이블에 삽입 (NaN 안전 처리)"""
    df = df.where(pd.notnull(df), None)  # NaN → None
    placeholders = ", ".join(["%s"] * len(df.columns))
    columns = ", ".join([f"`{col}`" for col in df.columns])
    sql = f"INSERT INTO `{table_name}` ({columns}) VALUES ({placeholders})"
    cursor.executemany(sql, df.values.tolist())
    conn.commit()

def process_csv_files():
    # SSH 터널 연결
    print("\n" + "="*60)
    print("SSH 터널 연결 중...")
    print("="*60)
    tunnel = SSHTunnelForwarder(**SSH_CONFIG)
    tunnel.start()
    print("✓ SSH 터널 연결 성공!")
    
    # MySQL 연결
    print("MySQL 서버 연결 중...")
    conn, cursor = connect_mysql(tunnel)
    print("✓ MySQL 서버 연결 성공!")
    
    print(f"데이터베이스 '{DB_NAME}' 선택 중...")
    cursor.execute(f"USE `{DB_NAME}`")
    print(f"✓ '{DB_NAME}' 선택 완료!\n")

    created_tables = []
    
    # CSV 파일 목록 확인
    csv_files = [f for f in os.listdir(CSV_DIR) if f.endswith(".csv")]
    total_files = len(csv_files)
    
    print(f"발견된 CSV 파일: {total_files}개\n")
    
    if total_files == 0:
        print("CSV 파일이 없습니다!")
        cursor.close()
        conn.close()
        tunnel.stop()
        return

    current_file = 0
    for file in csv_files:
        current_file += 1
        file_path = os.path.join(CSV_DIR, file)
        table_name = safe_table_name(os.path.splitext(file)[0])

        print(f"[{current_file}/{total_files}] {file} → 테이블명: {table_name}")

        try:
            # CSV 읽기
            df = pd.read_csv(file_path)
            
            # 모든 NaN 값을 "None" 문자열로 변환
            df = df.fillna("None")

            # 컬럼명 안전하게 처리 (강화된 nan 처리)
            new_columns = []
            for i, col in enumerate(df.columns, 1):
                if pd.isna(col):
                    new_columns.append(f"col_{i}")
                else:
                    col_str = str(col).strip()
                    if col_str == "" or col_str.lower() == "nan":
                        new_columns.append(f"col_{i}")
                    else:
                        cleaned = re.sub(r'\W+', '_', col_str)
                        if cleaned.lower() == "nan" or cleaned.lower().startswith("nan"):
                            new_columns.append(f"col_{i}")
                        else:
                            cleaned = cleaned.strip('_')
                            new_columns.append(cleaned if cleaned else f"col_{i}")
            df.columns = new_columns

            # 기존 테이블 삭제 (덮어쓰기 모드일 때)
            if OVERWRITE:
                cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
                print(f"  → 기존 테이블 삭제")

            # 테이블 생성
            create_table_from_csv(cursor, table_name, df)
            print(f"  → 테이블 생성 완료")

            # 데이터 삽입
            print(f"  → 데이터 삽입 중...")
            insert_data(cursor, conn, table_name, df)
            print(f"  → 데이터 커밋 중...")
            
            created_tables.append(table_name)
            print(f"  ✓ {len(df)}개 행 업로드 완료\n")

        except Exception as e:
            print(f"  ✗ 오류 발생: {e}\n")
            continue

    cursor.close()
    conn.close()
    tunnel.stop()

    print("="*60)
    print("=== 처리 완료 ===")
    print("="*60)
    print(f"업로드된 테이블 ({len(created_tables)}개):")
    for t in created_tables:
        print(f"  - {t}")
    print("="*60)

if __name__ == "__main__":
    process_csv_files()

c:\Users\wntjd\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\wntjd\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\wntjd\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,



SSH 터널 연결 중...
✓ SSH 터널 연결 성공!
MySQL 서버 연결 중...
✓ MySQL 서버 연결 성공!
데이터베이스 'etf1_db' 선택 중...
✓ 'etf1_db' 선택 완료!

발견된 CSV 파일: 1549개

[1/1549] BATS_A, 10.csv → 테이블명: BATS_A_10
  → 기존 테이블 삭제
  → 테이블 생성 완료
  → 데이터 삽입 중...
  → 데이터 커밋 중...
  ✓ 5154개 행 업로드 완료

[2/1549] BATS_A, 12M.csv → 테이블명: BATS_A_12M
  → 기존 테이블 삭제
  → 테이블 생성 완료
  → 데이터 삽입 중...
  → 데이터 커밋 중...
  ✓ 27개 행 업로드 완료

[3/1549] BATS_A, 1D.csv → 테이블명: BATS_A_1D
  → 기존 테이블 삭제
  → 테이블 생성 완료
  → 데이터 삽입 중...
  → 데이터 커밋 중...
  ✓ 4094개 행 업로드 완료

[4/1549] BATS_A, 1M.csv → 테이블명: BATS_A_1M
  → 기존 테이블 삭제
  → 테이블 생성 완료
  → 데이터 삽입 중...
  → 데이터 커밋 중...
  ✓ 312개 행 업로드 완료

[5/1549] BATS_A, 1W.csv → 테이블명: BATS_A_1W
  → 기존 테이블 삭제
  → 테이블 생성 완료
  → 데이터 삽입 중...
  → 데이터 커밋 중...
  ✓ 1355개 행 업로드 완료

[6/1549] BATS_A, 60.csv → 테이블명: BATS_A_60
  → 기존 테이블 삭제
  → 테이블 생성 완료
  → 데이터 삽입 중...
  → 데이터 커밋 중...
  ✓ 7615개 행 업로드 완료

[7/1549] BATS_AAPL, 10.csv → 테이블명: BATS_AAPL_10
  → 기존 테이블 삭제
  → 테이블 생성 완료
  → 데이터 삽입 중...
  → 데이터 커밋 중...
  ✓ 5154개 행 업로드 완료

[8/1549] B